In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
train = train_data.iloc[:, 3].values
label = train_data.iloc[:, 4].values
test = test_data.iloc[:, 3].values

In [ ]:
standard_error = train_data.iloc[:, 5].values
std_error = np.array(standard_error)
print(std_error.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(label, std_error)
plt.ylim([0.4,0.7])
plt.show()

In [ ]:
label = np.array(label)
train = np.array(train)
test = np.array(test)

print(label.shape)
print(train.shape)
print(test.shape)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import re
import pandas as pd

In [ ]:
def word_to_vec(inputs):
    tokenizer = Tokenizer(filters='', oov_token='<OOV>') # this tokenizer will filters nothing.
    tokenizer.fit_on_texts(inputs)

    tensor = tokenizer.texts_to_sequences(inputs)
    tensor = pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [ ]:
train_tensor, train_token = word_to_vec(train)

In [ ]:
max_len = train_tensor[0, :].shape[0]

In [ ]:
BUFFER_SIZE = len(train_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(train_tensor)//BATCH_SIZE
embedding_dim = 50
units = 60
vocab_train = len(train_token.word_index)+1

#dataset = tf.data.Dataset.from_tensor_slices((train_tensor, label)).shuffle(BUFFER_SIZE)
#dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()

        self.s = tf.keras.layers.Dense(units, kernel_regularizer=tf.keras.regularizers.L2(0.001))
        self.t = tf.keras.layers.Dense(units, kernel_regularizer=tf.keras.regularizers.L2(0.001))
        self.a = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L2(0.001))

    def call(self, query, value):
        #query: (None, 255, 1024)
        #value: (None, 1024)
        
        value = tf.expand_dims(value, 1)
        #query: (None, 255, 1024)
        #value: (1, 1024)
        
        score = self.a(tf.keras.activations.relu(self.s(query) + self.t(value)))
        
        attention_weights = tf.keras.activations.softmax(score, axis=1) 

        attention = attention_weights * value
        attention = tf.reduce_sum(attention, axis=1) 
            
        return attention, attention_weights

In [ ]:
attention = Attention(units)

In [ ]:
# encoder 
encoder = tf.keras.Input(shape=(max_len, ))
enc_embd = tf.keras.layers.Embedding(vocab_train, embedding_dim)(encoder)
encoder_gru = tf.keras.layers.GRU(units, return_sequences=True, return_state=True, kernel_regularizer=tf.keras.regularizers.L2(0.001))
output_e, hidden_e = encoder_gru(enc_embd)

# attention
attention_value, attention_weights = attention(output_e, hidden_e)

# concatenate the attention and hidden_e layer
context_vector = tf.concat([attention_value, hidden_e], axis=-1)

attention_layer = tf.keras.layers.Dense(context_vector.shape[1] , activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))
attention_ = attention_layer(context_vector)

# output
final_output = tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=tf.keras.regularizers.L2(0.001))
output_f = final_output(context_vector)


In [ ]:
model = tf.keras.Model(encoder, output_f)

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(train_tensor.shape, label.shape)

In [ ]:
model.fit(train_tensor, label, epochs=20, verbose=1)

In [ ]:
print(label[0:3])
print(model.predict(train_tensor[0:3]))

In [ ]:
test = train_token.texts_to_sequences(test)
test = pad_sequences(test, maxlen=max_len, padding='post')

In [ ]:
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
test_df['target'] = model.predict(test) #model.predict(padded_test)
print(test_df['target'])
test_df[['id','target']].to_csv('submission.csv', index=False)